In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import linear_model
from sklearn.model_selection import StratifiedKFold, KFold

from sklearn import metrics

In [2]:
#df = pd.read_excel("Residential-Building-Data-Set.xlsx", dtype = np.float)
#df = pd.read_excel("Residential-Building-Data-Set.xlsx")
df = pd.read_excel("Residential-Building-Data-Set - Copy.xlsx")

df.head(10)

,START YEAR,START QUARTER,COMPLETION YEAR,COMPLETION QUARTER,V-1,V-2,V-3,V-4,V-5,V-6,...,V-22.4,V-23.4,V-24.4,V-25.4,V-26.4,V-27.4,V-28.4,V-29.4,V-9,V-10
0,81,1,85,1,1,3150.0,920.0,598.5,190,1010.84,...,815.50,1755.00,8002.0,60.74,54.26,2978.26,41407.0,601988.1,2200,410
1,84,1,89,4,1,7600.0,1140.0,3040.0,400,963.81,...,1316.30,8364.78,8393.0,90.95,89.79,11379.37,44835.0,929027.1,5000,1000
2,78,1,81,4,1,4800.0,840.0,480.0,100,689.84,...,765.80,1755.00,4930.0,38.70,32.04,1653.06,37933.0,377828.6,1200,170
3,72,2,73,2,1,685.0,202.0,13.7,20,459.54,...,152.25,1442.31,1456.0,9.73,8.34,686.16,8194.0,122031.7,165,30
4,87,1,90,2,1,3000.0,800.0,1230.0,410,631.91,...,2244.10,9231.76,9286.0,136.60,140.20,9821.00,48260.0,1734973.5,5500,700
5,87,1,90,1,1,2500.0,640.0,1050.0,420,647.32,...,2244.10,9231.76,9286.0,136.60,140.20,9821.00,48260.0,1734973.5,5200,700
6,87,2,90,1,1,1810.0,492.0,1158.4,640,843.98,...,2526.40,9254.28,9306.0,136.56,138.80,9291.00,12393.0,1732937.5,5800,900
7,88,1,89,3,1,1150.0,380.0,575.0,500,590.68,...,3148.00,9248.40,9380.0,158.63,169.50,10082.00,49572.0,2318397.0,4600,600
8,76,3,77,4,1,2110.0,540.0,189.9,90,732.14,...,550.40,1755.00,4252.0,30.13,21.54,2134.49,18409.0,402158.1,870,110
9,80,1,80,4,1,3030.0,930.0,515.1,170,1007.38,...,764.50,1755.00,8420.0,54.62,44.79,2206.19,40234.0,629797.2,1700,190


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Columns: 109 entries, START YEAR to V-10
dtypes: float64(94), int64(15)
memory usage: 316.9 KB


In [4]:
df.dtypes

START YEAR              int64
START QUARTER           int64
COMPLETION YEAR         int64
COMPLETION QUARTER      int64
V-1                     int64
                       ...   
V-27.4                float64
V-28.4                float64
V-29.4                float64
V-9                     int64
V-10                    int64
Length: 109, dtype: object

In [5]:
df.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372 entries, 0 to 371
Data columns (total 109 columns):
 #   Column              Dtype  
---  ------              -----  
 0   START YEAR          int64  
 1   START QUARTER       int64  
 2   COMPLETION YEAR     int64  
 3   COMPLETION QUARTER  int64  
 4   V-1                 int64  
 5   V-2                 float64
 6   V-3                 float64
 7   V-4                 float64
 8   V-5                 int64  
 9   V-6                 float64
 10  V-7                 int64  
 11  V-8                 int64  
 12  V-11                float64
 13  V-12                float64
 14  V-13                float64
 15  V-14                float64
 16  V-15                float64
 17  V-16                float64
 18  V-17                float64
 19  V-18                float64
 20  V-19                float64
 21  V-20                int64  
 22  V-21                float64
 23  V-22                float64
 24  V-23                float64
 25

In [6]:
# drop the columns that are not needed to form the input variable
input_n = df.drop(['V-9', 'V-10'], axis='columns')

In [7]:
print(input_n)

     START YEAR  START QUARTER  COMPLETION YEAR  COMPLETION QUARTER  V-1  \
0            81              1               85                   1    1   
1            84              1               89                   4    1   
2            78              1               81                   4    1   
3            72              2               73                   2    1   
4            87              1               90                   2    1   
..          ...            ...              ...                 ...  ...   
367          83              4               86                   1   20   
368          81              2               82                   4   20   
369          84              4               86                   3   20   
370          82              3               84                   4   20   
371          80              1               81                   3   20   

        V-2     V-3     V-4  V-5      V-6  ...  V-20.4       V-21.4   V-22.4  \
0    31

In [8]:
# target variable is V-9 which is the actual sales price
target_n = df['V-9']

In [9]:
print(target_n)

0      2200
1      5000
2      1200
3       165
4      5500
       ... 
367    1100
368     740
369     860
370    1100
371     460
Name: V-9, Length: 372, dtype: int64


## Linear Regression Model Using StratifiedKFold Cross Validation

1. First we creat a modle with all the features:

In [10]:
kfold = KFold(n_splits=10)
m_lr = linear_model.LinearRegression() 

i = 1

mae =[]
mape =[]
mse =[]
rmse =[]

In [11]:
#write a function for MAPE, MAPE built in function current not available
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

x = input_n
y = target_n
for train,test in kfold.split(x,y):
    model = m_lr.fit(x.iloc[train],y.iloc[train])
    #pred_prob = model.predict_proba(x.iloc[test])
    pred = model.predict(x.iloc[test])

    #prediction = m_rfc.fit(x[train],y[train]).predict_proba(x[test])
    #fpr, tpr, t = roc_curve(y.iloc[test], pred_prob[:, 1])
    mae.append(metrics.mean_absolute_error(y.iloc[test], pred))
    
    #metrics.mean_absolute_percentage_error will be provided in scikit-learn 0.24
    #mape.append(metrics.mean_absolute_percentage_error(y.iloc[test], pred))    
    mape_temp = mean_absolute_percentage_error(y.iloc[test], pred)
    mape.append(mape_temp)
    
    mse.append(metrics.mean_squared_error(y.iloc[test], pred))
    
    #tprs.append(np.interp(mean_fpr, fpr, tpr))
    #roc_auc = sklearn.metrics.auc(fpr, tpr)
    #aucs.append(roc_auc)
    #plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))
    i= i+1
    

# plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
# mean_tpr = np.mean(tprs, axis=0)
# mean_auc = sklearn.metrics.auc(mean_fpr, mean_tpr)
# plt.plot(mean_fpr, mean_tpr, color='blue',
#          label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)

# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC')
# plt.legend(loc="lower right")
# plt.show()



Report performance scores: MAE, MAPE, RMSE

In [12]:
print(mae)
print(mape)
print(mse)

[240.7384717399803, 130.59996896166732, 213.3770898326307, 83.93219429762797, 81.6665664981633, 66.21843629125986, 67.85384329789717, 92.57934443288428, 83.02283804517076, 161.07516546000022]
[7.516342360128707, 13.749389362584441, 13.74456689921571, 10.165869533583468, 8.41499711270285, 15.129399250035041, 13.15954940172645, 13.672254899128914, 16.510549382687188, 17.511995926992714]
[180018.18748161546, 36708.47232822179, 373726.2794716626, 14410.412515899045, 10746.075662932599, 13399.586737548541, 6738.474776072329, 14554.592362868605, 12205.926273396679, 113996.9845163351]


In [13]:
rmse = np.sqrt(mse)
print(rmse)

[424.28550232 191.59455193 611.33156263 120.04337764 103.66328021
 115.75658399  82.08821338 120.64241527 110.48043389 337.63439475]


2. Next we creat a modle with only 10 features: We pick V-1 to V-8 and V-12 and V-13 since we interpreted the data and we think they are the most relevant variables.

In [14]:
# drop the columns that are not needed to form the input variable
input_n2 = df[['V-1', 'V-2','V-3','V-4','V-5','V-6','V-7','V-8','V-12','V-13']]

In [15]:
print(input_n2)

     V-1     V-2     V-3     V-4  V-5      V-6  V-7   V-8   V-12    V-13
0      1  3150.0   920.0   598.5  190  1010.84   16  1200   56.2   61.52
1      1  7600.0  1140.0  3040.0  400   963.81   23  2900  106.0  103.03
2      1  4800.0   840.0   480.0  100   689.84   15   630   41.0   41.25
3      1   685.0   202.0    13.7   20   459.54    4   140   12.1   10.03
4      1  3000.0   800.0  1230.0  410   631.91   13  5000  203.8  162.84
..   ...     ...     ...     ...  ...      ...  ...   ...    ...     ...
367   20  1350.0   350.0   108.0   80   251.37    9   830  103.0  101.84
368   20   600.0   150.0    36.0   60   299.55    6   570   59.1   65.61
369   20  1900.0   430.0   285.0  150   364.41    7   640  120.2  107.20
370   20   510.0   160.0    30.6   60   245.28    9   790   76.9   79.93
371   20   890.0   230.0    35.6   40   237.03    6   350   51.3   56.13

[372 rows x 10 columns]


In [16]:
print(target_n)

0      2200
1      5000
2      1200
3       165
4      5500
       ... 
367    1100
368     740
369     860
370    1100
371     460
Name: V-9, Length: 372, dtype: int64


In [17]:
m_lr2 = linear_model.LinearRegression() 

i = 1

mae2 =[]
mape2 =[]
mse2 =[]
rmse2 =[]

In [18]:
#write a function for MAPE, MAPE built in function current not available
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

x = input_n2
y = target_n
for train,test in kfold.split(x,y):
    model = m_lr2.fit(x.iloc[train],y.iloc[train])
    pred = model.predict(x.iloc[test])

    mae2.append(metrics.mean_absolute_error(y.iloc[test], pred))
    
    #metrics.mean_absolute_percentage_error will be provided in scikit-learn 0.24
    #mape.append(metrics.mean_absolute_percentage_error(y.iloc[test], pred))    
    mape2_temp = mean_absolute_percentage_error(y.iloc[test], pred)
    mape2.append(mape2_temp)
    
    mse2.append(metrics.mean_squared_error(y.iloc[test], pred))
    i= i+1

Report performance scores: MAE, MAPE, RMSE

In [19]:
print(mae2)
print(mape2)
print(mse2)

[308.16990861333534, 181.55779494179612, 164.07585809834538, 108.81163583598136, 135.48825494957345, 76.23893594797796, 103.3406803844553, 92.13994532120466, 83.45702563294404, 142.6131916573637]
[11.33928817134207, 13.13554299621689, 9.483290660175758, 10.549923070202198, 10.114618530198193, 11.227822711355172, 16.99890185977104, 14.847060831435119, 18.86300291132466, 14.520181898698068]
[232596.8358549356, 121252.6624340827, 72837.65199375855, 30252.293713935607, 39026.579477153275, 8815.09851484257, 16014.447798415531, 12103.439390510499, 10437.833558292077, 39354.882903197045]


In [20]:
rmse2 = np.sqrt(mse2)
print(rmse)

[424.28550232 191.59455193 611.33156263 120.04337764 103.66328021
 115.75658399  82.08821338 120.64241527 110.48043389 337.63439475]
